In [3]:
import os
import pandas as pd
import numpy as np
import pytz

fname = "../data_generated/Patients_and_Weather_extended.csv"
average_considered_hours = [4,8,12,24]
weather_columns = ['temp','humidity','pressure','wind_speed']

# Read the csv files
weather_df = pd.read_csv("../data/weather_data.csv")
patients_df = pd.read_csv("../data/U4H_RawMonitoringData_from_20160623.csv")
patients_df = patients_df[patients_df['Region'].notnull()] # Remove a few last lines

# Find the weather date and time and save them separately
weather_df['dt_weather'] = weather_df['dt'].apply(lambda i: pd.Timestamp(i, unit='s', tz='CET'))
weather_df['weather_only_date'] = [d.date() for d in weather_df['dt_weather']]
weather_df['weather_only_time'] = [d.time() for d in weather_df['dt_weather']]

# Find the patient date and time and save them separately
patients_df['patients_only_date'] = patients_df['Assess_Date_With_Time'].apply(lambda i: i.split()[0])
patients_df['patients_only_time'] = patients_df['Assess_Date_With_Time'].apply(lambda i: i.split()[1])

# Find the weather date and time and save them separately
patients_df['patients_only_date'] = pd.to_datetime(patients_df['patients_only_date']).apply(lambda i: i.date())
patients_df['patients_only_time'] = pd.to_datetime(patients_df['patients_only_time']).apply(lambda i: i.time())

### Merge the file to create a csv with the headers combined
patients_and_weather_df = patients_df.merge(weather_df, how='left', left_on=['patients_only_date'], right_on=['weather_only_date'])


# =========================== STEP 1. Connect to closest weather data (just like before)
patients_and_weather_df["time_difference"] = patients_and_weather_df.apply(lambda x: np.abs(
                                               (x['patients_only_time'].hour - x['weather_only_time'].hour)*3600+
                                               (x['patients_only_time'].minute - x['weather_only_time'].minute)*60+
                                               (x['patients_only_time'].second - x['weather_only_time'].second)),
                                               axis=1)

pawdf_grouped = patients_and_weather_df.groupby(by=["Merida ID", "Assess_Date_With_Time"])

def select_lowest_time_diff(x):
    index = np.where(x["time_difference"] == x["time_difference"].min())[0][0]
    return x.iloc[index, :]

patients_and_weather_df = pawdf_grouped.apply(select_lowest_time_diff)
patients_and_weather_df.to_csv(fname)

In [30]:
import datetime

print('==================== STEP 2 STARTED =====================================')
patients_and_weather_df['Assess_Date_With_Time'] = pd.to_datetime(patients_and_weather_df['Assess_Date_With_Time'],
                                                                  format='%Y-%m-%d %H:%M:%S')
patients_and_weather_df['Assess_Date_With_Time'] = patients_and_weather_df['Assess_Date_With_Time']
patients_and_weather_df['Assess_date_No_Time'] = patients_and_weather_df['Assess_Date_With_Time'].dt.date
for c in weather_columns:
    for h in average_considered_hours:
        patients_and_weather_df[c+"_avg"+str(h)+"h"] = 0

for index, row in patients_and_weather_df.iterrows():
    print("Row: ", index)
    for h in average_considered_hours:
        weather_subselection = weather_df[
            (row['Assess_Date_With_Time'] >= weather_df['dt_weather']) &
            ((row['Assess_Date_With_Time'] - weather_df['dt_weather']) <= datetime.timedelta(hours=h))]
        # print(weather_subselection[['dt_weather']+weather_columns])
        for c in weather_columns:
            patients_and_weather_df.loc[index,c+"_avg"+str(h)+"h"] = np.nanmean(weather_subselection[c])
patients_and_weather_df.to_csv(fname)

==================== STEP 2 STARTED =====================================
Row:  ('1000405', '2015-01-07 11:14:31.000')
Row:  ('1000405', '2015-01-08 09:47:05.000')
Row:  ('1000405', '2015-01-09 09:28:19.000')
Row:  ('1000405', '2015-01-12 09:17:07.000')
Row:  ('1000405', '2015-01-13 09:25:21.000')
Row:  ('1000405', '2015-01-14 09:38:09.000')
Row:  ('1000405', '2015-01-14 09:38:10.000')
Row:  ('1000405', '2015-01-14 09:39:24.000')
Row:  ('1000405', '2015-01-15 09:35:12.000')
Row:  ('1000405', '2015-01-16 09:28:43.000')
Row:  ('1000405', '2015-01-17 10:26:11.000')
Row:  ('1000405', '2015-01-18 10:12:56.000')
Row:  ('1000405', '2015-01-19 09:49:32.000')
Row:  ('1000405', '2015-01-19 09:49:33.000')
Row:  ('1000405', '2015-01-20 09:37:45.000')
Row:  ('1000405', '2015-01-20 09:41:45.000')
Row:  ('1000405', '2015-01-21 09:26:54.000')
Row:  ('1000405', '2015-01-22 09:30:56.000')
Row:  ('1000405', '2015-01-23 09:54:47.000')
Row:  ('1000405', '2015-01-24 07:54:37.000')
Row:  ('1000405', '2015-01

Row:  ('1000602', '2015-03-13 08:10:16.000')
Row:  ('1000602', '2015-03-14 08:16:50.000')
Row:  ('1000602', '2015-03-15 08:22:45.000')
Row:  ('1000602', '2015-03-16 08:14:58.000')
Row:  ('1000602', '2015-03-17 08:09:58.000')
Row:  ('1000602', '2015-03-18 08:15:50.000')
Row:  ('1000602', '2015-03-19 08:11:29.000')
Row:  ('1000602', '2015-03-20 08:07:43.000')
Row:  ('1000602', '2015-03-21 08:25:39.000')
Row:  ('1000602', '2015-03-21 08:26:21.000')
Row:  ('1000602', '2015-03-21 08:26:25.000')
Row:  ('1000602', '2015-03-22 08:20:41.000')
Row:  ('1000602', '2015-03-23 08:19:00.000')
Row:  ('1000602', '2015-03-24 08:08:17.000')
Row:  ('1000602', '2015-03-25 08:12:13.000')
Row:  ('1000602', '2015-03-26 08:21:27.000')
Row:  ('1000602', '2015-03-27 08:15:46.000')
Row:  ('1000602', '2015-03-27 08:18:09.000')
Row:  ('1000602', '2015-03-27 08:18:10.000')
Row:  ('1000602', '2015-03-28 08:06:36.000')
Row:  ('1000602', '2015-03-29 08:14:46.000')
Row:  ('1000602', '2015-03-30 08:17:48.000')
Row:  ('10

Row:  ('1001534', '2015-06-18 09:27:11.000')
Row:  ('1001534', '2015-06-19 09:05:51.000')
Row:  ('1001534', '2015-06-20 09:13:48.000')
Row:  ('1001534', '2015-06-22 08:50:43.000')
Row:  ('1001534', '2015-06-23 09:19:54.000')
Row:  ('1001534', '2015-06-24 09:32:05.000')
Row:  ('1001534', '2015-06-25 08:45:48.000')
Row:  ('1001534', '2015-06-29 08:39:02.000')
Row:  ('1001534', '2015-06-29 08:39:03.000')
Row:  ('1001534', '2015-06-29 08:39:04.000')
Row:  ('1001534', '2015-06-30 09:30:43.000')
Row:  ('1001534', '2015-07-01 09:32:09.000')
Row:  ('1001534', '2015-07-02 08:50:56.000')
Row:  ('1001534', '2015-07-03 09:32:05.000')
Row:  ('1001534', '2015-07-04 09:38:50.000')
Row:  ('1001534', '2015-07-06 09:44:57.000')
Row:  ('1001534', '2015-07-07 09:09:45.000')
Row:  ('1001534', '2015-07-08 09:20:08.000')
Row:  ('1001534', '2015-07-09 09:06:38.000')
Row:  ('1001534', '2015-07-10 09:37:33.000')
Row:  ('1001534', '2015-07-11 09:25:44.000')
Row:  ('1001534', '2015-07-13 14:33:47.000')
Row:  ('10

Row:  ('1001534', '2016-02-10 09:34:20.000')
Row:  ('1001534', '2016-02-11 10:06:40.000')
Row:  ('1001534', '2016-02-11 10:08:42.000')
Row:  ('1001534', '2016-02-12 09:39:28.000')
Row:  ('1001534', '2016-02-13 10:47:25.000')
Row:  ('1001534', '2016-02-22 09:01:26.000')
Row:  ('1001534', '2016-02-23 10:18:29.000')
Row:  ('1001534', '2016-02-24 09:31:46.000')
Row:  ('1001534', '2016-02-25 10:15:16.000')
Row:  ('1001534', '2016-02-26 09:30:48.000')
Row:  ('1001534', '2016-02-27 10:53:02.000')
Row:  ('1001534', '2016-02-29 10:03:18.000')
Row:  ('1001534', '2016-03-01 11:17:46.000')
Row:  ('1001534', '2016-03-02 09:00:53.000')
Row:  ('1001534', '2016-03-03 10:23:33.000')
Row:  ('1001534', '2016-03-04 08:19:27.000')
Row:  ('1001534', '2016-03-05 10:15:23.000')
Row:  ('1001534', '2016-03-07 09:14:30.000')
Row:  ('1001534', '2016-03-10 09:21:24.000')
Row:  ('1001534', '2016-03-10 09:21:25.000')
Row:  ('1001534', '2016-03-11 10:38:33.000')
Row:  ('1001534', '2016-03-12 10:45:24.000')
Row:  ('10

Row:  ('1002016', '2015-06-11 08:59:33.000')
Row:  ('1002016', '2015-06-12 08:57:58.000')
Row:  ('1002016', '2015-06-13 09:03:10.000')
Row:  ('1002016', '2015-06-14 09:01:20.000')
Row:  ('1002016', '2015-06-15 09:10:21.000')
Row:  ('1002016', '2015-06-15 12:23:21.000')
Row:  ('1002016', '2015-06-15 12:23:22.000')
Row:  ('1002016', '2015-06-16 09:01:14.000')
Row:  ('1002016', '2015-06-17 08:43:57.000')
Row:  ('1002016', '2015-06-18 09:02:00.000')
Row:  ('1002016', '2015-06-19 08:56:22.000')
Row:  ('1002279', '2014-07-17 12:30:00.000')
Row:  ('1002279', '2014-07-21 08:44:23.000')
Row:  ('1002279', '2014-07-22 08:53:14.000')
Row:  ('1002279', '2014-07-23 09:18:06.000')
Row:  ('1002279', '2014-07-24 09:10:17.000')
Row:  ('1002279', '2014-07-25 09:05:07.000')
Row:  ('1002279', '2014-07-25 12:50:44.000')
Row:  ('1002279', '2014-07-28 09:56:36.000')
Row:  ('1002279', '2014-07-28 09:56:37.000')
Row:  ('1002279', '2014-07-28 13:05:13.000')
Row:  ('1002279', '2014-07-28 13:05:14.000')
Row:  ('10

...

Row:  ('1017566', '2016-01-28 14:07:12.000')
Row:  ('1017566', '2016-01-29 10:10:49.000')
Row:  ('1017566', '2016-01-30 09:16:22.000')
Row:  ('1017566', '2016-01-31 09:24:36.000')
Row:  ('1017566', '2016-02-01 09:55:33.000')
Row:  ('1017566', '2016-02-03 09:58:33.000')
Row:  ('1017566', '2016-02-03 09:58:35.000')
Row:  ('1017566', '2016-02-03 09:59:37.000')
Row:  ('1017566', '2016-02-04 09:35:06.000')
Row:  ('1017566', '2016-02-05 09:39:18.000')
Row:  ('1017566', '2016-02-06 09:57:17.000')
Row:  ('1017566', '2016-02-07 09:49:05.000')
Row:  ('1017566', '2016-02-08 09:40:58.000')
Row:  ('1017566', '2016-02-09 09:36:42.000')
Row:  ('1017566', '2016-02-10 11:04:12.000')
Row:  ('1017566', '2016-02-10 11:04:13.000')
Row:  ('1017566', '2016-02-10 11:04:14.000')
Row:  ('1017566', '2016-02-10 11:05:02.000')
Row:  ('1017566', '2016-02-10 11:05:03.000')
Row:  ('1017566', '2016-02-11 09:45:29.000')
Row:  ('1017566', '2016-02-11 09:45:30.000')
Row:  ('1017566', '2016-02-11 09:45:31.000')
Row:  ('10

Row:  ('9900001', '2014-06-27 09:07:53.000')
Row:  ('9900001', '2014-06-27 09:10:37.000')
Row:  ('9900001', '2014-06-27 09:10:41.000')
Row:  ('9900001', '2014-06-27 09:14:32.000')
Row:  ('9900001', '2014-06-27 09:14:35.000')
Row:  ('9900001', '2014-06-27 09:14:36.000')
Row:  ('9900001', '2014-06-27 09:23:10.000')
Row:  ('9900001', '2014-06-27 09:23:11.000')
Row:  ('9900001', '2014-06-27 09:23:15.000')
Row:  ('9900001', '2014-06-28 09:31:37.000')
Row:  ('9900001', '2014-06-28 09:32:22.000')
Row:  ('9900001', '2014-06-28 09:32:23.000')
Row:  ('9900001', '2014-06-29 10:21:19.000')
Row:  ('9900001', '2014-06-30 09:03:30.000')
Row:  ('9900001', '2014-07-01 09:00:46.000')
Row:  ('9900001', '2014-07-02 09:59:58.000')
Row:  ('9900001', '2014-07-03 09:27:13.000')
Row:  ('9900001', '2014-07-04 09:01:03.000')
Row:  ('9900001', '2014-07-05 09:18:22.000')
Row:  ('9900001', '2014-07-06 12:50:08.000')
Row:  ('9900001', '2014-07-07 08:46:19.000')
Row:  ('9900001', '2014-07-08 09:33:58.000')
Row:  ('99

Row:  ('9900006', '2014-06-03 12:57:52.000')
Row:  ('9900006', '2014-06-04 16:29:11.000')
Row:  ('9900006', '2014-06-05 15:20:26.000')
Row:  ('9900006', '2014-06-06 15:03:15.000')
Row:  ('9900006', '2014-06-06 15:03:57.000')
Row:  ('9900006', '2014-06-06 15:03:58.000')
Row:  ('9900006', '2014-06-10 13:09:49.000')
Row:  ('9900006', '2014-06-11 11:36:38.000')
Row:  ('9900006', '2014-06-12 09:26:37.000')
Row:  ('9900006', '2014-06-13 14:31:50.000')
Row:  ('9900006', '2014-06-13 14:32:33.000')
Row:  ('9900006', '2014-06-13 14:32:34.000')
Row:  ('9900006', '2014-06-14 12:23:34.000')
Row:  ('9900006', '2014-06-15 11:16:27.000')
Row:  ('9900006', '2014-06-16 12:03:27.000')
Row:  ('9900006', '2014-06-17 15:29:13.000')
Row:  ('9900006', '2014-06-18 14:28:05.000')
Row:  ('9900006', '2014-06-19 11:49:19.000')
Row:  ('9900006', '2014-06-20 11:22:59.000')
Row:  ('9900006', '2014-06-25 12:05:14.000')
Row:  ('9900006', '2014-06-25 12:09:09.000')
Row:  ('9900006', '2014-06-26 13:06:54.000')
Row:  ('99

Row:  ('9900009', '2014-08-11 12:18:12.000')
Row:  ('9900009', '2014-08-12 08:25:54.000')
Row:  ('9900009', '2014-08-13 08:18:17.000')
Row:  ('9900009', '2014-08-14 07:37:35.000')
Row:  ('9900009', '2014-08-15 07:52:19.000')
Row:  ('9900009', '2014-08-16 08:20:40.000')
Row:  ('9900009', '2014-08-17 07:55:25.000')
Row:  ('9900009', '2014-08-18 09:24:54.000')
Row:  ('9900009', '2014-08-19 19:36:54.000')
Row:  ('9900009', '2014-08-20 09:07:00.000')
Row:  ('9900009', '2014-08-21 20:08:27.000')
Row:  ('9900009', '2014-08-22 09:06:31.000')
Row:  ('9900009', '2014-08-23 10:11:13.000')
Row:  ('9900009', '2014-08-24 09:38:05.000')
Row:  ('9900009', '2014-08-26 09:33:56.000')
Row:  ('9900009', '2014-08-27 07:44:27.000')
Row:  ('9900009', '2014-08-28 07:56:20.000')
Row:  ('9900009', '2014-08-29 07:10:36.000')
Row:  ('9900009', '2014-08-30 09:39:31.000')
Row:  ('9900009', '2014-08-31 10:03:41.000')
Row:  ('9900009', '2014-09-01 11:19:56.000')
Row:  ('9900009', '2014-09-02 08:48:18.000')
Row:  ('99

Row:  ('9900125', '2015-10-05 08:53:09.000')
Row:  ('9900125', '2015-10-05 14:43:11.000')
Row:  ('9900125', '2015-10-05 14:43:46.000')
Row:  ('9900125', '2016-02-25 09:29:21.000')
Row:  ('9900125', '2016-02-25 09:29:22.000')
Row:  ('9900125', '2016-02-25 09:29:23.000')
Row:  ('9900125', '2016-02-25 09:29:24.000')
Row:  ('9900125', '2016-02-25 09:29:25.000')
Row:  ('9900125', '2016-02-25 09:58:34.000')
Row:  ('9900125', '2016-02-25 09:58:35.000')
Row:  ('9900125', '2016-04-05 10:59:49.000')
Row:  ('9900125', '2016-04-25 12:16:17.000')
Row:  ('9900125', '2016-05-18 10:10:49.000')
Row:  ('9900125', '2016-05-26 14:15:07.000')
Row:  ('9900127', '2015-06-15 17:41:15.000')
Row:  ('9900127', '2015-07-16 15:00:33.000')
Row:  ('9900127', '2015-07-27 08:27:24.000')
Row:  ('9900127', '2015-07-27 08:28:53.000')
Row:  ('9900127', '2015-08-13 09:17:58.000')
Row:  ('9900127', '2015-08-13 09:17:59.000')
Row:  ('9900127', '2015-08-13 09:18:00.000')
Row:  ('9900127', '2015-08-13 09:18:59.000')
Row:  ('99

In [27]:
patients_and_weather_df[[i for i in patients_and_weather_df.columns if ('temp' in i)or('humidity' in i)or('pressure' in i)]]

temp    temp_min    temp_max  \
Merida ID Assess_Date_With_Time                                         
1000405   2015-01-07 11:14:31.000  275.812333  275.812333  275.812333   
          2015-01-08 09:47:05.000  277.034500  277.034500  277.034500   
          2015-01-09 09:28:19.000  274.768667  274.768667  274.768667   
          2015-01-12 09:17:07.000  277.809333  277.809333  277.809333   
          2015-01-13 09:25:21.000  275.274333  275.274333  275.274333   
          2015-01-14 09:38:09.000  274.214000  274.214000  274.214000   
          2015-01-14 09:38:10.000  274.214000  274.214000  274.214000   
          2015-01-14 09:39:24.000  274.214000  274.214000  274.214000   
          2015-01-15 09:35:12.000  277.124000  277.124000  277.124000   
          2015-01-16 09:28:43.000  277.026667  277.026667  277.026667   
          2015-01-17 10:26:11.000  275.226000  275.226000  275.226000   
          2015-01-18 10:12:56.000  273.333000  273.333000  273.333000   
          2015-01-19 09:49:32.000  270.426500  270.426500  270.426500   
          2015-01-19 09:49:33.000  270.426500  270.426500  270.426500   
          2015-01-20 09:37:45.000  273.298000  273.298000  273.298000   
          2015-01-20 09:41:45.000  273.298000  273.298000  273.298000   
          2015-01-21 09:26:54.000  272.629000  272.629000  272.629000   
          2015-01-22 09:30:56.000  269.410500  269.410500  269.410500   
          2015-01-23 09:54:47.000  272.754500  272.754500  272.754500   
          2015-01-24 07:54:37.000  274.806333  274.806333  274.806333   
          2015-01-25 08:17:12.000  273.636000  273.636000  273.636000   
          2015-01-26 09:34:07.000  276.938500  276.938500  276.938500   
          2015-01-27 09:20:07.000  273.873000  273.873000  273.873000   
          2015-01-28 09:27:03.000  277.017000  277.017000  277.017000   
          2015-01-29 08:56:31.000  274.237333  274.237333  274.237333   
          2015-01-30 09:29:54.000  273.787333  273.787333  273.787333   
          2015-01-31 09:02:47.000  272.593000  272.593000  272.593000   
          2015-02-01 10:44:20.000  268.150667  268.150667  268.150667   
          2015-02-02 09:32:15.000  269.827500  269.827500  269.827500   
          2015-02-03 09:02:57.000  264.453333  264.453333  264.453333   
...                                       ...         ...         ...   
9900127   2015-08-13 09:18:59.000  285.480000  283.150000  288.150000   
          2015-08-13 10:23:52.000  288.230000  287.590000  289.150000   
          2015-08-13 10:23:53.000  288.230000  287.590000  289.150000   
          2015-08-13 10:25:28.000  288.230000  287.590000  289.150000   
          2015-08-17 12:57:30.000  294.470000  291.150000  299.260000   
          2015-08-17 12:57:31.000  294.470000  291.150000  299.260000   
          2015-08-17 12:57:55.000  294.470000  291.150000  299.260000   
          2015-08-17 13:41:19.000  294.120000  292.150000  296.480000   
          2015-08-17 13:57:41.000  294.120000  292.150000  296.480000   
          2015-09-09 14:03:11.000  292.570000  290.150000  295.370000   
          2015-09-09 15:10:03.000  293.680000  290.150000  298.710000   
          2015-09-18 11:14:18.000  288.150000  288.150000  288.150000   
          2015-09-18 11:14:19.000  288.150000  288.150000  288.150000   
          2015-09-18 11:14:20.000  288.150000  288.150000  288.150000   
          2015-09-23 20:13:37.000  285.770000  283.710000  287.150000   
          2015-09-24 08:21:09.000  285.300000  284.820000  286.150000   
          2015-10-20 14:10:32.000  282.066124  282.066124  282.066124   
          2015-11-02 10:54:20.000  284.175391  284.175391  284.175391   
          2016-03-30 09:38:54.000  276.150000  276.150000  276.150000   
          2016-03-31 13:59:56.000  279.150000  279.150000  279.150000   
          2016-03-31 13:59:57.000  279.150000  279.150000  279.150000   
          2016-03-31 14:01:54.000  279.150000  279.150000  279.150000   
          2016-03-31 14:04

In [32]:
print('==================== STEP 3 STARTED =====================================')
interesting_parameters = ("pulse_Triage", "spo2_Triage", "question_Triage", "manual_Triage")
num_days = 10
for par in interesting_parameters:
    patients_and_weather_df[par + "_" + str(par)] = 0

for index, row in patients_and_weather_df.iterrows():
    print("Row: ", index)
    for d in range(1,num_days):
        subselection = patients_and_weather_df[(patients_and_weather_df['Merida ID'] == row['Merida ID']) &
            ((row['Assess_date_No_Time'] - patients_and_weather_df['Assess_date_No_Time']) < datetime.timedelta(days=d+1)) &
            ((row['Assess_date_No_Time'] - patients_and_weather_df['Assess_date_No_Time']) > datetime.timedelta(days=d-1))]
        #print("\t", len(subselection))
        for par in interesting_parameters:
            patients_and_weather_df.loc[index, par+"_"+str(d)] = np.nanmean(subselection[par])

==================== STEP 3 STARTED =====================================
Row:  ('1000405', '2015-01-07 11:14:31.000')


E:\Tools\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice
  from ipykernel import kernelapp as app


Row:  ('1000405', '2015-01-08 09:47:05.000')
Row:  ('1000405', '2015-01-09 09:28:19.000')
Row:  ('1000405', '2015-01-12 09:17:07.000')
Row:  ('1000405', '2015-01-13 09:25:21.000')
Row:  ('1000405', '2015-01-14 09:38:09.000')
Row:  ('1000405', '2015-01-14 09:38:10.000')
Row:  ('1000405', '2015-01-14 09:39:24.000')
Row:  ('1000405', '2015-01-15 09:35:12.000')
Row:  ('1000405', '2015-01-16 09:28:43.000')
Row:  ('1000405', '2015-01-17 10:26:11.000')
Row:  ('1000405', '2015-01-18 10:12:56.000')
Row:  ('1000405', '2015-01-19 09:49:32.000')
Row:  ('1000405', '2015-01-19 09:49:33.000')
Row:  ('1000405', '2015-01-20 09:37:45.000')
Row:  ('1000405', '2015-01-20 09:41:45.000')
Row:  ('1000405', '2015-01-21 09:26:54.000')
Row:  ('1000405', '2015-01-22 09:30:56.000')
Row:  ('1000405', '2015-01-23 09:54:47.000')
Row:  ('1000405', '2015-01-24 07:54:37.000')
Row:  ('1000405', '2015-01-25 08:17:12.000')
Row:  ('1000405', '2015-01-26 09:34:07.000')
Row:  ('1000405', '2015-01-27 09:20:07.000')
Row:  ('10

Row:  ('1000602', '2015-03-13 08:10:16.000')
Row:  ('1000602', '2015-03-14 08:16:50.000')
Row:  ('1000602', '2015-03-15 08:22:45.000')
Row:  ('1000602', '2015-03-16 08:14:58.000')
Row:  ('1000602', '2015-03-17 08:09:58.000')
Row:  ('1000602', '2015-03-18 08:15:50.000')
Row:  ('1000602', '2015-03-19 08:11:29.000')
Row:  ('1000602', '2015-03-20 08:07:43.000')
Row:  ('1000602', '2015-03-21 08:25:39.000')
Row:  ('1000602', '2015-03-21 08:26:21.000')
Row:  ('1000602', '2015-03-21 08:26:25.000')
Row:  ('1000602', '2015-03-22 08:20:41.000')
Row:  ('1000602', '2015-03-23 08:19:00.000')
Row:  ('1000602', '2015-03-24 08:08:17.000')
Row:  ('1000602', '2015-03-25 08:12:13.000')
Row:  ('1000602', '2015-03-26 08:21:27.000')
Row:  ('1000602', '2015-03-27 08:15:46.000')
Row:  ('1000602', '2015-03-27 08:18:09.000')
Row:  ('1000602', '2015-03-27 08:18:10.000')
Row:  ('1000602', '2015-03-28 08:06:36.000')
Row:  ('1000602', '2015-03-29 08:14:46.000')
Row:  ('1000602', '2015-03-30 08:17:48.000')
Row:  ('10

Row:  ('1001534', '2015-06-16 08:33:24.000')
Row:  ('1001534', '2015-06-17 09:34:05.000')
Row:  ('1001534', '2015-06-18 09:27:11.000')
Row:  ('1001534', '2015-06-19 09:05:51.000')
Row:  ('1001534', '2015-06-20 09:13:48.000')
Row:  ('1001534', '2015-06-22 08:50:43.000')
Row:  ('1001534', '2015-06-23 09:19:54.000')
Row:  ('1001534', '2015-06-24 09:32:05.000')
Row:  ('1001534', '2015-06-25 08:45:48.000')
Row:  ('1001534', '2015-06-29 08:39:02.000')
Row:  ('1001534', '2015-06-29 08:39:03.000')
Row:  ('1001534', '2015-06-29 08:39:04.000')
Row:  ('1001534', '2015-06-30 09:30:43.000')
Row:  ('1001534', '2015-07-01 09:32:09.000')
Row:  ('1001534', '2015-07-02 08:50:56.000')
Row:  ('1001534', '2015-07-03 09:32:05.000')
Row:  ('1001534', '2015-07-04 09:38:50.000')
Row:  ('1001534', '2015-07-06 09:44:57.000')
Row:  ('1001534', '2015-07-07 09:09:45.000')
Row:  ('1001534', '2015-07-08 09:20:08.000')
Row:  ('1001534', '2015-07-09 09:06:38.000')
Row:  ('1001534', '2015-07-10 09:37:33.000')
Row:  ('10

...

Row:  ('1017434', '2016-02-23 09:14:10.000')
Row:  ('1017434', '2016-02-24 10:47:38.000')
Row:  ('1017434', '2016-02-25 13:55:59.000')
Row:  ('1017434', '2016-02-26 08:22:27.000')
Row:  ('1017434', '2016-02-27 11:48:50.000')
Row:  ('1017434', '2016-02-29 08:34:36.000')
Row:  ('1017434', '2016-03-01 10:19:00.000')
Row:  ('1017434', '2016-03-01 11:47:21.000')
Row:  ('1017434', '2016-03-01 11:47:22.000')
Row:  ('1017434', '2016-03-02 10:06:49.000')
Row:  ('1017434', '2016-03-03 07:58:55.000')
Row:  ('1017434', '2016-03-04 08:29:48.000')
Row:  ('1017434', '2016-03-07 08:00:44.000')
Row:  ('1017434', '2016-03-08 10:08:04.000')
Row:  ('1017434', '2016-03-08 11:28:22.000')
Row:  ('1017434', '2016-03-08 11:28:23.000')
Row:  ('1017434', '2016-03-09 09:50:12.000')
Row:  ('1017434', '2016-03-10 09:17:51.000')
Row:  ('1017434', '2016-03-11 11:24:05.000')
Row:  ('1017434', '2016-03-12 10:43:06.000')
Row:  ('1017434', '2016-03-14 09:56:47.000')
Row:  ('1017434', '2016-03-15 09:19:53.000')
Row:  ('10

Row:  ('1017753', '2016-03-15 09:45:50.000')
Row:  ('1017753', '2016-03-16 09:29:52.000')
Row:  ('1017753', '2016-03-16 09:51:00.000')
Row:  ('1017753', '2016-03-16 09:51:01.000')
Row:  ('1017753', '2016-03-16 09:51:02.000')
Row:  ('1017753', '2016-03-16 09:51:03.000')
Row:  ('1017753', '2016-03-16 09:51:04.000')
Row:  ('1017753', '2016-03-17 09:29:48.000')
Row:  ('1017753', '2016-03-18 09:24:08.000')
Row:  ('1017753', '2016-03-19 08:58:19.000')
Row:  ('1017753', '2016-03-19 09:29:34.000')
Row:  ('1017753', '2016-03-19 09:29:35.000')
Row:  ('1017753', '2016-03-19 09:29:36.000')
Row:  ('1017753', '2016-03-19 09:29:37.000')
Row:  ('1017753', '2016-03-19 09:29:38.000')
Row:  ('1017753', '2016-03-20 11:48:57.000')
Row:  ('1017753', '2016-03-21 09:25:17.000')
Row:  ('1017753', '2016-03-22 09:39:54.000')
Row:  ('1017753', '2016-03-23 09:46:08.000')
Row:  ('1017753', '2016-03-29 09:27:27.000')
Row:  ('1017753', '2016-03-30 09:45:26.000')
Row:  ('1017753', '2016-03-31 09:42:14.000')
Row:  ('10

Row:  ('9900003', '2014-07-02 11:50:32.000')
Row:  ('9900003', '2014-07-03 10:36:05.000')
Row:  ('9900003', '2014-07-04 11:24:44.000')
Row:  ('9900003', '2014-07-05 12:18:49.000')
Row:  ('9900003', '2014-07-06 11:40:19.000')
Row:  ('9900003', '2014-07-07 10:56:46.000')
Row:  ('9900003', '2014-07-08 11:23:07.000')
Row:  ('9900003', '2014-07-09 10:47:03.000')
Row:  ('9900003', '2014-07-10 10:50:32.000')
Row:  ('9900003', '2014-07-11 10:42:38.000')
Row:  ('9900003', '2014-07-12 17:42:32.000')
Row:  ('9900003', '2014-07-13 11:37:00.000')
Row:  ('9900003', '2014-07-14 10:41:46.000')
Row:  ('9900003', '2014-07-15 11:49:25.000')
Row:  ('9900003', '2014-07-16 10:41:08.000')
Row:  ('9900003', '2014-07-17 11:14:19.000')
Row:  ('9900003', '2014-07-18 10:49:51.000')
Row:  ('9900003', '2014-07-19 13:14:16.000')
Row:  ('9900003', '2014-07-20 11:44:55.000')
Row:  ('9900003', '2014-07-21 10:24:14.000')
Row:  ('9900003', '2014-07-22 10:38:13.000')
Row:  ('9900003', '2014-07-23 10:51:27.000')
Row:  ('99

Row:  ('9900007', '2014-06-02 09:23:45.000')
Row:  ('9900007', '2014-06-03 09:38:05.000')
Row:  ('9900007', '2014-06-04 09:31:42.000')
Row:  ('9900007', '2014-06-05 13:07:01.000')
Row:  ('9900007', '2014-06-06 09:41:52.000')
Row:  ('9900007', '2014-06-07 12:00:16.000')
Row:  ('9900007', '2014-06-08 12:01:55.000')
Row:  ('9900007', '2014-06-09 11:11:57.000')
Row:  ('9900007', '2014-06-10 09:36:20.000')
Row:  ('9900007', '2014-06-11 05:51:57.000')
Row:  ('9900007', '2014-06-11 09:40:30.000')
Row:  ('9900007', '2014-06-12 09:32:36.000')
Row:  ('9900007', '2014-06-13 09:30:47.000')
Row:  ('9900007', '2014-06-14 09:34:26.000')
Row:  ('9900007', '2014-06-15 13:37:29.000')
Row:  ('9900007', '2014-06-16 13:30:50.000')
Row:  ('9900007', '2014-06-18 08:40:26.000')
Row:  ('9900007', '2014-06-19 08:38:48.000')
Row:  ('9900007', '2014-06-20 11:36:52.000')
Row:  ('9900007', '2014-06-21 09:42:31.000')
Row:  ('9900007', '2014-06-23 14:46:57.000')
Row:  ('9900007', '2014-06-23 14:47:55.000')
Row:  ('99

Row:  ('9900123', '2015-08-14 11:06:09.000')
Row:  ('9900123', '2015-08-14 14:56:35.000')
Row:  ('9900123', '2015-08-14 14:58:01.000')
Row:  ('9900123', '2015-08-14 14:58:02.000')
Row:  ('9900123', '2015-08-14 15:00:42.000')
Row:  ('9900123', '2015-08-14 15:00:43.000')
Row:  ('9900123', '2015-08-14 15:02:57.000')
Row:  ('9900123', '2015-08-14 15:07:52.000')
Row:  ('9900123', '2015-08-14 15:08:39.000')
Row:  ('9900123', '2015-08-14 15:08:40.000')
Row:  ('9900123', '2015-08-14 15:09:04.000')
Row:  ('9900123', '2015-08-21 13:48:01.000')
Row:  ('9900124', '2014-10-28 08:58:44.000')
Row:  ('9900124', '2014-10-28 09:00:30.000')
Row:  ('9900124', '2014-10-29 10:11:39.000')
Row:  ('9900124', '2014-10-30 12:44:54.000')
Row:  ('9900124', '2014-10-30 13:42:54.000')
Row:  ('9900124', '2014-11-05 14:31:23.000')
Row:  ('9900124', '2014-11-11 09:22:18.000')
Row:  ('9900124', '2014-11-12 13:18:22.000')
Row:  ('9900124', '2014-11-12 13:33:13.000')
Row:  ('9900124', '2014-11-12 14:09:17.000')
Row:  ('99

In [33]:
patients_and_weather_df.to_csv(fname)